In [1]:
from bs4 import BeautifulSoup
import requests
import lxml
import json
import csv
import time
import random
import os
from datetime import datetime
from collections import Counter



from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
import time
import random
import pickle


In [2]:
url = "https://shop.casio.ru/catalog/filter/gender-is-male/apply/"
headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }

r = requests.get(url=url, headers=headers)
if not os.path.exists("data"):
    os.mkdir("data")

with open("data/page_1.html", "w", encoding="utf-8") as f:
    f.write(r.text)
    

In [3]:
url = "https://shop.casio.ru/catalog/filter/gender-is-male/apply/"
headers = {
        "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/94.0.4606.81 Safari/537.36"
    }


with open("data/page_1.html", encoding="utf-8") as f:
    src = f.read()
    
soup = BeautifulSoup(src, "lxml")
pages_count = int(soup.find("div", class_="bx-pagination-container").find_all("a")[-2].text)

for i in range(1, pages_count + 1):
    url = f"https://shop.casio.ru/catalog/filter/gender-is-male/apply/?PAGEN_1={i}"
    
    r = requests.get(url=url, headers=headers)
    
    with open(f"data/page_{i}.html", "w", encoding="utf-8") as f:
              f.write(r.text)
    time.sleep(2)

In [14]:
cur_date = datetime.now().strftime("%d_%m_%Y")

with open(f"data_{cur_date}.csv", "w", encoding="utf-8") as file:
    writer = csv.writer(file)
    
    writer.writerow(
        (
            "Артикул",
            "Цена",
            "Ссылка"
        )
    )

data = []

for page in range(1, pages_count + 1):
    with open(f"data/page_{page}.html", encoding="utf-8") as f:
        src = f.read()
    
    soup = BeautifulSoup(src, "lxml")
    item_cards = soup.find_all("a", class_="product-item__link")
    
    for item in item_cards:
        product_article = item.find("p", class_="product-item__articul").text.strip()
        product_price = item.find("p", class_="product-item__price").text.lstrip("руб. ")
        product_url = "https://shop.casio.ru" + item.get("href")
        
        data.append(
            {
                "product_article": product_article,
                "product_price": product_price,
                "product_url": product_url
            }
        )
        
        with open(f"data_{cur_date}.csv", "a", encoding="utf-8") as file:
            writer = csv.writer(file)
    
            writer.writerow(
                (
                    product_article,
                    product_price,
                    product_url
                )
            )
        
        
        #print(f"Article: {product_article}\nPrice: {product_price}\nURL: {product_url}\n########")
    print(f"[INFO] Обработана страница {page}/8")

  
with open(f"data_{cur_date}.json", "a", encoding="utf-8") as file:
    json.dump(data, file, indent=4, ensure_ascii=False)

[INFO] Обработана страница 1/8
[INFO] Обработана страница 2/8
[INFO] Обработана страница 3/8
[INFO] Обработана страница 4/8
[INFO] Обработана страница 5/8
[INFO] Обработана страница 6/8
[INFO] Обработана страница 7/8
[INFO] Обработана страница 8/8


In [10]:
len(data)

230

In [15]:
import pandas as pd
df = pd.read_csv('data_16_10_2021.csv')
df

,Артикул,Цена,Ссылка
0,GA-700SKE-7AER,12 390,https://shop.casio.ru/catalog/g-shock-youth/ga...
1,GA-2100SKE-7AER,11 790,https://shop.casio.ru/catalog/g-shock-youth/ga...
2,DW-5600SKE-7ER,11 790,https://shop.casio.ru/catalog/g-shock/dw-5600s...
3,GA-110SKE-8AER,12 390,https://shop.casio.ru/catalog/g-shock-youth/ga...
4,GA-900SKE-8AER,12 390,https://shop.casio.ru/catalog/g-shock-youth/ga...
...,...,...,...
225,PRG-330-1E,20 690,https://shop.casio.ru/catalog/protrek/prg_330_1e/
226,PRT-B50-1ER,20 690,https://shop.casio.ru/catalog/protrek-connecte...
227,PRT-B50-4ER,20 690,https://shop.casio.ru/catalog/protrek-connecte...
228,PRW-30-1AER,28 840,https://shop.casio.ru/catalog/protrek/prw_30_1...
